In [1]:
# generate data for testing acc@10
# pick the first 100 ligands from the test_set
# pair all proteins with all ligands
# (100x100 pairs)
# (total of 10,000 pairs...)

In [2]:
from htmd.ui import *
from htmd.molecule.voxeldescriptors import *
from pathlib import Path
import numpy as np
import pickle as pk
from multiprocessing import Pool
import pandas as pd

%load_ext autoreload
%autoreload 2

/home/alex/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/
To update: conda update htmd -c acellera -c psi4

You are on the latest HTMD version (1.13.9).



In [8]:
test_size = 100
base_dest = 'processed_data/test_acc10'
Path(base_dest).mkdir(exist_ok=True, parents=True)
df = pd.read_csv('test_lig_pro_pairs.csv')
idxs = sorted(list(df.lig_id.unique()))
idxs = idxs[:test_size]

In [ ]:
dests = []
ligs = []
proteins = []
score = []

for lig_idx in idxs:
    lig_src = 'processed_data/ligands/' + str(lig_idx).zfill(4) + '.pk'
    with open(lig_src, 'rb') as f:
        lig_features, centers = pk.load(f)
    
    for pro_idx in idxs:
        mol = Molecule('./training_data/'+str(pro_idx).zfill(4)+'_pro_cg.pdb', keepaltloc='all')
        pro_features, _ = getVoxelDescriptors(mol, usercenters=np.array(centers), voxelsize=1, method='NUMBA')
        pro_features = pro_features.reshape(24, 24, 24, pro_features.shape[1])
        combined = np.concatenate((pro_features, lig_features), axis=3)
    
        dest = base_dest + '/' + str(pro_idx).zfill(4) + '_pro_' + str(lig_idx).zfill(4) + '_lig.npy'
        
        with open(dest, 'wb') as f:
            np.save(f, combined)
            
        dests.append(dests)
        ligs.append(lig_idx)
        proteins.append(pro_idx)
        score.append(1 if lig_idx == pro_idx else 0)
        
        
csv_dest = 'test_acc10.csv'
pro_lig_record = pd.DataFrame({'lig_id': ligs, 'pro_id': proteins, 'dests': dests, 'score': score}, index=None)
pro_lig_record.to_csv(csv_dest, index=None)

/home/alex/anaconda3/lib/python3.6/site-packages/htmd/molecule/readers.py:200: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if np.issubdtype(mol._dtypes[field], np.float) \
